In [1]:
!pip install data-refinery 
!pip install pyspark

The directory '/home/cesargallego/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/cesargallego/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Could not find a version that satisfies the requirement data-refinery (from versions: )
No matching distribution found for data-refinery
The directory '/home/cesargallego/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/cesargallego/.cache/pip' or its parent directory is not owned by the current us

In [5]:
from datarefinery.tuple.TupleDSL import compose
from datarefinery.TupleOperations import append, substitution, wrap
from datarefinery.FieldOperations import date_parser, explode_date, remove_columns, add_column_prefix, column_category
import pyspark

ImportError: No module named 'datarefinery'

In [ ]:
weather_categories = ['cold', 'warm', 'hot']

row = {'id': 1, 'fecha': '2017-02-23', 'weather': 'cold'}

# Local

In [ ]:
complete_date = compose(
        date_parser(["%Y-%m-%d"]),
        explode_date,
        remove_columns("hour", "minute", "second"),
        add_column_prefix("x")
)

operation = Tr(append(['weather'], column_category(weather_categories))).then(append(['fecha'], complete_date)).apply()

(inp, out, err) = operation(row)

print('Input: ', inp)
print('Ouput: ', out)

# Cluster

In [ ]:
sc = pyspark.SparkContext('local[*]')

In [ ]:
weather = sc.parallelize([row])
out_rdd = weather.map(operation).map(lambda x: x[1])

print(out_rdd.collect())